In [7]:
import numpy as np
from scipy.io import loadmat
from sklearn.neighbors import NearestNeighbors

In [8]:
# Load all of the saved files
train_feat = np.loadtxt('train_feat.gzip', dtype = 'float')
gallery_feat = np.loadtxt('gallery_feat.gzip', dtype = 'float')
query_feat = np.loadtxt('query_feat.gzip', dtype = 'float')
val_feat = np.loadtxt('val_feat.gzip', dtype = 'float')
train_noval_feat = np.loadtxt('train_noval_feat.gzip', dtype = 'float')
train_labels = np.loadtxt('train_labels.gzip', dtype = 'uint16')
gallery_labels = np.loadtxt('gallery_labels.gzip', dtype = 'uint16')
query_labels = np.loadtxt('query_labels.gzip', dtype = 'uint16')
val_labels = np.loadtxt('val_labels.gzip', dtype = 'uint16')
train_noval_labels = np.loadtxt('train_noval_labels.gzip', dtype = 'uint16')
#train_filelist = np.loadtxt('train_filelist.gzip', dtype = 'uint16')
#gallery_filelist = np.loadtxt('gallery_filelist.gzip', dtype = 'uint16')
#query_filelist = np.loadtxt('query_filelist.gzip', dtype = 'uint16')
#val_filelist = np.loadtxt('val_filelist.gzip', dtype = 'uint16')
#train_noval_filelist = np.loadtxt('train_noval_filelist.gzip', dtype = 'uint16')
train_camId = np.loadtxt('train_camId.gzip', dtype = 'uint16') # uint8
gallery_camId = np.loadtxt('gallery_camId.gzip', dtype = 'uint16')
query_camId = np.loadtxt('query_camId.gzip', dtype = 'uint16')
val_camId = np.loadtxt('val_camId.gzip', dtype = 'uint16')
train_noval_camId = np.loadtxt('train_noval_camId.gzip', dtype = 'uint16')

In [21]:
print (gallery_labels, gallery_labels.shape)
print (gallery_camId, gallery_camId.shape)

a = np.ones((gallery_labels.shape))*query_camId[0] # This vector is for comparison, has all of the elements as the query camID.
b = np.ones((gallery_labels.shape))*gallery_labels[0] # This vector stores the first item's gallery label

checking = gallery_camId == a
checking_2 = gallery_labels == b


print (a, a.shape)
print (b, b.shape)
print (checking, chacking.shape)
print ()

[   3    3    3 ... 1463 1463 1463] (5328,)
[1 1 1 ... 2 2 2] (5328,)
[1. 1. 1. ... 1. 1. 1.] (5328,)
[3. 3. 3. ... 3. 3. 3.] (5328,)


In [3]:
# Remove the pictures from the gallery that have the same label and are taken by the same camera as out FIRST PICTURE
# in query array
a = np.ones((gallery_labels.shape[0]))*query_camId[0]
print (a, a.shape)
b = np.ones((gallery_labels.shape[0]))*query_labels[0]
print (b, b.shape)
print (gallery_camId, gallery_camId.shape, gallery_labels, gallery_labels.shape)
c = gallery_camId == a
d = gallery_labels == b
print (c, d, c.shape, d.shape)
e = np.logical_and(c, d)
e=np.logical_not(e)
print (e, e.shape)

gallery_feat_removed = gallery_feat[e, :]
gallery_labels_removed = gallery_feat[e, :]


#gallery_feat_removed = gallery_feat[(not(gallery_camId==a and gallery_labels==b)), :]
#gallery_labels_removed = gallery_feat[(not(gallery_camId==query_camId[0] and gallery_label==query_labels[0])), :]

k = 5

nbrs = NearestNeighbors(n_neighbors=k).fit(gallery_feat_removed)
distances, indices = nbrs.kneighbors(query_feat[:1, :])

print ( query_feat[:1, :], query_feat[:1, :].shape)
print(indices, indices.shape)
print (distances, distances.shape)

well_predicted = gallery_labels_removed[indices, :] == np.ones((k))*query_labels[0]
precision_or_recall = np.sum(well_predicted)/k
print (precision_or_recall)

[1. 1. 1. ... 1. 1. 1.] (5328,)
[3. 3. 3. ... 3. 3. 3.] (5328,)
[1 1 1 ... 2 2 2] (5328,) [   3    3    3 ... 1463 1463 1463] (5328,)
[ True  True  True ... False False False] [ True  True  True ... False False False] (5328,) (5328,)
[False False False ...  True  True  True] (5328,)
[[0.94088095 0.6120016  0.19601382 ... 0.03830115 0.05461279 0.01004573]] (1, 2048)
[[5046 2773 5216  148 5049]] (1, 5)
[[15.60366779 16.41445879 17.13820046 17.34235112 17.47453528]] (1, 5)
0.0


/Users/karolyhorvath/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [4]:
print (query_feat.dtype, query_feat.max())
print (gallery_feat, gallery_feat.shape)
print (query_feat, query_feat.shape)

float64 8.214414596557617
[[1.05525982 0.29310364 0.19821346 ... 0.00684856 0.0561235  0.01862506]
 [0.98361254 0.33195475 0.26549727 ... 0.02350602 0.08491972 0.00514889]
 [0.82696682 0.39545193 0.13419056 ... 0.02507348 0.06063982 0.03505545]
 ...
 [0.01961832 0.17617127 0.54575312 ... 0.33026204 0.93925601 0.56220758]
 [0.05719076 0.60635394 0.58300596 ... 0.40994745 1.37061644 0.55975795]
 [0.07245111 0.51656747 0.59082419 ... 0.60488421 1.05610693 0.72939718]] (5328, 2048)
[[0.94088095 0.6120016  0.19601382 ... 0.03830115 0.05461279 0.01004573]
 [1.55147386 0.35008368 0.29907167 ... 0.         0.38377187 0.10629545]
 [0.09531084 0.35792962 0.1479917  ... 0.25905719 0.34150648 0.1575031 ]
 ...
 [0.69368488 0.91065705 1.7122643  ... 0.60383898 1.46147764 0.23456964]
 [0.33765692 1.02798986 0.42934412 ... 0.32115731 1.51256549 0.3386271 ]
 [0.04796965 0.49429518 0.61193639 ... 0.74304843 1.20724857 0.66419029]] (1400, 2048)


In [5]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors([[0, 1], [0,0]])
print(indices)
print (distances)

[[3 4 0 1 5]
 [3 0 1 4 2]]
[[1.         2.         2.23606798 2.82842712 3.16227766]
 [1.41421356 1.41421356 2.23606798 2.23606798 3.60555128]]


In [6]:
k = 5
nbrs = NearestNeighbors(n_neighbors=k).fit(gallery_feat)
distances, indices = nbrs.kneighbors(query_feat)
print(indices, indices.shape)
print (distances, distances.shape)

[[   1    2    0 5049 2776]
 [   5    4    6    3 1969]
 [  10    7    8    9   14]
 ...
 [5320 5317 5319 5318 5194]
 [5321 5323 5322 5327 5326]
 [5327 5326 5324 5325 5323]] (1400, 5)
[[ 6.09014114  6.14945446  7.54184004 15.60366779 16.41445879]
 [ 8.31251325  8.88552532  9.87390657 14.0570714  15.82024625]
 [ 6.906436    7.25947872  7.6402917   8.14847465 11.88007043]
 ...
 [ 6.22842337  7.20920733 10.5651047  11.62377886 14.30915442]
 [ 8.43849631  9.50885541 12.01978502 14.03768135 15.21211206]
 [ 4.91326734  8.28096591  9.0167629  11.23723619 14.29231764]] (1400, 5)
